In [43]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from underthesea import word_tokenize, sent_tokenize
import re
from gensim.parsing.preprocessing import remove_stopwords
import gensim.downloader as api
from gensim.models import Word2Vec

In [44]:
with open('../lecture3/sentiment.txt') as f:
    sentiment = f.readlines()
    
text_processing = []
for i in sentiment:
    text_processing.append(i.strip().lower().split('\t'))

In [45]:
data = []
label = []

for i in range(len(text_processing)):
    data.append(re.sub(r'[^\w\s]', '', text_processing[i][0].strip()))
    label.append(int(text_processing[i][1]))
    

In [50]:
# remove stop words and tokenization
data_processed = []
for i in data:
    data_processed.append(re.split("\W+", i))
    # data_processed.append(re.split("\W+", remove_stopwords(i)))
    

In [51]:
data_processed

[['wow', 'loved', 'this', 'place'],
 ['crust', 'is', 'not', 'good'],
 ['not', 'tasty', 'and', 'the', 'texture', 'was', 'just', 'nasty'],
 ['stopped',
  'by',
  'during',
  'the',
  'late',
  'may',
  'bank',
  'holiday',
  'off',
  'rick',
  'steve',
  'recommendation',
  'and',
  'loved',
  'it'],
 ['the',
  'selection',
  'on',
  'the',
  'menu',
  'was',
  'great',
  'and',
  'so',
  'were',
  'the',
  'prices'],
 ['now',
  'i',
  'am',
  'getting',
  'angry',
  'and',
  'i',
  'want',
  'my',
  'damn',
  'pho'],
 ['honeslty', 'it', 'didnt', 'taste', 'that', 'fresh'],
 ['the',
  'potatoes',
  'were',
  'like',
  'rubber',
  'and',
  'you',
  'could',
  'tell',
  'they',
  'had',
  'been',
  'made',
  'up',
  'ahead',
  'of',
  'time',
  'being',
  'kept',
  'under',
  'a',
  'warmer'],
 ['the', 'fries', 'were', 'great', 'too'],
 ['a', 'great', 'touch'],
 ['service', 'was', 'very', 'prompt'],
 ['would', 'not', 'go', 'back'],
 ['the',
  'cashier',
  'had',
  'no',
  'care',
  'what',


In [152]:
# w2v_model = api.load("word2vec-google-news-300")

In [64]:
# Tạo và đào tạo mô hình Word2Vec
model = Word2Vec(sentences=data_processed, vector_size=300, window=5, min_count=1, sg=0)

# Lưu mô hình
model.save("word2vec.model")


In [65]:

# Đọc mô hình đã lưu
w2v_model = Word2Vec.load("word2vec.model")

In [66]:
w2v_model.wv["loved"].shape

(300,)

In [74]:
w2v_model.wv.most_similar('very')

[('and', 0.9983127117156982),
 ('the', 0.9982776045799255),
 ('a', 0.9982678294181824),
 ('to', 0.9982270002365112),
 ('i', 0.9981989860534668),
 ('was', 0.9980222582817078),
 ('for', 0.9979893565177917),
 ('this', 0.9979819059371948),
 ('is', 0.9979233145713806),
 ('but', 0.9979103803634644)]

In [55]:
word2vector = []

for words in data_processed:
    count = 0
    convert = np.empty(300)
    print(words)
    for word in words:
        convert += w2v_model.wv[word]
        count += 1
    print(count)
    word2vector.append(convert / count)

['wow', 'loved', 'this', 'place']
4
['crust', 'is', 'not', 'good']
4
['not', 'tasty', 'and', 'the', 'texture', 'was', 'just', 'nasty']
8
['stopped', 'by', 'during', 'the', 'late', 'may', 'bank', 'holiday', 'off', 'rick', 'steve', 'recommendation', 'and', 'loved', 'it']
15
['the', 'selection', 'on', 'the', 'menu', 'was', 'great', 'and', 'so', 'were', 'the', 'prices']
12
['now', 'i', 'am', 'getting', 'angry', 'and', 'i', 'want', 'my', 'damn', 'pho']
11
['honeslty', 'it', 'didnt', 'taste', 'that', 'fresh']
6
['the', 'potatoes', 'were', 'like', 'rubber', 'and', 'you', 'could', 'tell', 'they', 'had', 'been', 'made', 'up', 'ahead', 'of', 'time', 'being', 'kept', 'under', 'a', 'warmer']
22
['the', 'fries', 'were', 'great', 'too']
5
['a', 'great', 'touch']
3
['service', 'was', 'very', 'prompt']
4
['would', 'not', 'go', 'back']
4
['the', 'cashier', 'had', 'no', 'care', 'what', 'so', 'ever', 'on', 'what', 'i', 'had', 'to', 'say', 'it', 'still', 'ended', 'up', 'being', 'wayyy', 'overpriced']
21
[

<ipython-input-55-7da7dbfde4f7>:8: RuntimeWarning: invalid value encountered in add
  convert += w2v_model.wv[word]


In [56]:
word2vector = np.array(word2vector)

In [77]:
word2vector.shape

(1000, 300)

In [78]:
X_train, X_test, y_train, y_test = train_test_split(word2vector, label, test_size=0.2, random_state=8)

In [79]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.4, random_state=8)

In [80]:
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)
y_val = np.array(y_val).reshape(-1, 1)


In [81]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((800, 300), (800, 1), (120, 300), (120, 1), (80, 300), (80, 1))

In [82]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(300,)),
    tf.keras.layers.Dense(units=100, activation='relu'),
    tf.keras.layers.Dense(units=1, activation="sigmoid")
])

In [83]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [84]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


/home/uoa/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/framework/constant_op.py:102: RuntimeWarning: overflow encountered in cast
  return ops.EagerTensor(value, ctx.device_name, dtype)


25/25 [==============================] - 1s 8ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.4833
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.4833
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.4833
Epoch 4/10
25/25 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.4833
Epoch 5/10
25/25 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.4833
Epoch 6/10
25/25 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.4833
Epoch 7/10
25/25 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.4833
Epoch 8/10
25/25 [==============================] - 0s 4ms/step - loss: n